<a href="https://colab.research.google.com/github/Chinmay-47/Autism_Detection/blob/main/Autism_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification of Autism Behaviour using Random Forest Classifier in Sci-kit Learn

#### Library needed to read arff files in the dataset
Import the library and load the dataset

In [ ]:
!pip3 install liac-arff

In [ ]:
import arff
with open('Autism-Child-Data.arff', 'r') as f:
    dataset = arff.load(f)

In [ ]:
data = dataset['data']

#### Exploring the Dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(data,columns=['A1_Score','A2_Score','A3_Score','A4_Score','A5_Score','A6_Score','A7_Score','A8_Score','A9_Score','A10_Score','Age','Gender','ethnicity','Jaundice','Autism_in_fam_members','Country','used_app_before','Result','age_desc','relation','Class'])

In [134]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Age,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class
0,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO
1,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO
4,1,1,1,1,1,1,1,1,1,1,5.0,m,yes,no,no,YES
6,1,0,1,1,1,1,0,1,0,1,5.0,m,no,no,no,YES
7,1,1,1,1,1,1,1,1,0,0,5.0,f,no,no,no,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,7.0,f,yes,yes,no,YES
288,1,0,0,0,1,0,1,0,0,1,7.0,f,yes,yes,no,NO
289,1,0,1,1,1,1,1,0,0,1,4.0,m,no,no,no,YES
290,1,1,1,0,1,1,1,1,1,1,4.0,m,no,no,no,YES


##### Drop rows with empty values

In [ ]:
df = df.dropna()

##### Check to see if the classes are balanced

In [ ]:
df.groupby("Class")["Class"].count()

Class
NO     122
YES    126
Name: Class, dtype: int64

##### Explore other variables

In [ ]:
df.groupby("ethnicity")["ethnicity"].count()

ethnicity
Asian               46
Black               14
Hispanic             7
Latino               8
Middle Eastern      26
Others              14
Pasifika             2
South Asian         21
Turkish              2
White-European     108
Name: ethnicity, dtype: int64

In [ ]:
df.groupby("Country")["Country"].count()

Country
Afghanistan               2
Argentina                 1
Armenia                   3
Australia                23
Austria                   1
Bahrain                   2
Bangladesh                6
Bhutan                    1
Brazil                    2
Bulgaria                  1
Canada                    7
Costa Rica                1
Egypt                     2
Europe                    1
Georgia                   2
Germany                   1
Ghana                     1
India                    42
Iraq                      2
Ireland                   1
Isle of Man               1
Italy                     2
Japan                     1
Jordan                    9
Libya                     1
Malaysia                  1
Malta                     1
Mexico                    2
Nepal                     1
Netherlands               1
New Zealand              13
Nigeria                   1
Oman                      1
Pakistan                  1
Philippines               4
Romania     

In [ ]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Age,Gender,ethnicity,Jaundice,Autism_in_fam_members,Country,used_app_before,Result,age_desc,relation,Class
0,1,1,0,0,1,1,0,1,0,0,6.0,m,Others,no,no,Jordan,no,5.0,4-11 years,Parent,NO
1,1,1,0,0,1,1,0,1,0,0,6.0,m,Middle Eastern,no,no,Jordan,no,5.0,4-11 years,Parent,NO
4,1,1,1,1,1,1,1,1,1,1,5.0,m,Others,yes,no,United States,no,10.0,4-11 years,Parent,YES
6,1,0,1,1,1,1,0,1,0,1,5.0,m,White-European,no,no,United Kingdom,no,7.0,4-11 years,Parent,YES
7,1,1,1,1,1,1,1,1,0,0,5.0,f,Middle Eastern,no,no,Bahrain,no,8.0,4-11 years,Parent,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,7.0,f,White-European,yes,yes,United Kingdom,no,10.0,4-11 years,Parent,YES
288,1,0,0,0,1,0,1,0,0,1,7.0,f,White-European,yes,yes,Australia,no,4.0,4-11 years,Parent,NO
289,1,0,1,1,1,1,1,0,0,1,4.0,m,Latino,no,no,Brazil,no,7.0,4-11 years,Parent,YES
290,1,1,1,0,1,1,1,1,1,1,4.0,m,South Asian,no,no,India,no,9.0,4-11 years,Parent,YES


#### Drop columns that we do not want for training our model

In [ ]:
df = df.drop("Country",axis=1)
df = df.drop("ethnicity",axis=1)
df = df.drop("age_desc",axis=1)
df = df.drop("relation",axis=1)
df = df.drop("Result",axis = 1)

In [ ]:
df

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Age,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class
0,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO
1,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO
4,1,1,1,1,1,1,1,1,1,1,5.0,m,yes,no,no,YES
6,1,0,1,1,1,1,0,1,0,1,5.0,m,no,no,no,YES
7,1,1,1,1,1,1,1,1,0,0,5.0,f,no,no,no,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,7.0,f,yes,yes,no,YES
288,1,0,0,0,1,0,1,0,0,1,7.0,f,yes,yes,no,NO
289,1,0,1,1,1,1,1,0,0,1,4.0,m,no,no,no,YES
290,1,1,1,0,1,1,1,1,1,1,4.0,m,no,no,no,YES


#### One Hot Encode the Cateorical Variables

In [ ]:
temp = pd.get_dummies(df["Age"])
temp

,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0
6,0,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
287,0,0,0,1,0,0,0,0
288,0,0,0,1,0,0,0,0
289,1,0,0,0,0,0,0,0
290,1,0,0,0,0,0,0,0


##### Add the one Hot Encoded Variables into the dataset and drop the old categorical variables

In [ ]:
data = pd.concat([df,temp],axis=1,join='outer')
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Age,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,6.0,m,no,no,no,NO,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,5.0,m,yes,no,no,YES,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,5.0,m,no,no,no,YES,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,5.0,f,no,no,no,YES,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,7.0,f,yes,yes,no,YES,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,7.0,f,yes,yes,no,NO,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,4.0,m,no,no,no,YES,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,4.0,m,no,no,no,YES,1,0,0,0,0,0,0,0


In [ ]:
data =data.drop("Age",axis=1)

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,m,no,no,no,NO,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,m,no,no,no,NO,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,m,yes,no,no,YES,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,m,no,no,no,YES,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,f,no,no,no,YES,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,f,yes,yes,no,YES,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,f,yes,yes,no,NO,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,m,no,no,no,YES,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,m,no,no,no,YES,1,0,0,0,0,0,0,0


##### Convert different boolean variables to binary variables

In [ ]:
data['Gender'] = df['Gender'].replace(['m','f'],['1','0'])

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,1,no,no,no,NO,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,1,no,no,no,NO,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,yes,no,no,YES,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,1,no,no,no,YES,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,no,no,no,YES,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,0,yes,yes,no,YES,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,0,yes,yes,no,NO,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,1,no,no,no,YES,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,1,no,no,no,YES,1,0,0,0,0,0,0,0


In [ ]:
data['Jaundice'] = df['Jaundice'].replace(['yes','no'],['1','0'])

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,used_app_before,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,1,0,no,no,NO,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,1,0,no,no,NO,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,no,no,YES,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,1,0,no,no,YES,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,no,no,YES,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,0,1,yes,no,YES,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,0,1,yes,no,NO,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,1,0,no,no,YES,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,1,0,no,no,YES,1,0,0,0,0,0,0,0


In [ ]:
data = data.drop("used_app_before",axis=1)

In [ ]:
data['Autism_in_fam_members'] = df['Autism_in_fam_members'].replace(['yes','no'],['1','0'])

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,1,0,0,NO,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,1,0,0,NO,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,0,YES,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,1,0,0,YES,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,YES,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,0,1,1,YES,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,0,1,1,NO,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,1,0,0,YES,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,1,0,0,YES,1,0,0,0,0,0,0,0


In [ ]:
data['Class'] = df['Class'].replace(['YES','NO'],['1','0'])

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,Class,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,0,0,0,0,0,0,0


#### Separate the class labels from the training features

In [ ]:
y_labels = data["Class"].to_list()

In [ ]:
y_labels

['0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0'

###### Sanity Check the labels

In [ ]:
print(list(set(y_labels)))

['1', '0']


In [ ]:
data = data.drop("Class",axis=1)

In [ ]:
data

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,Gender,Jaundice,Autism_in_fam_members,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
0,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0
1,1,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0
6,1,0,1,1,1,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,0,0,0
288,1,0,0,0,1,0,1,0,0,1,0,1,1,0,0,0,1,0,0,0,0
289,1,0,1,1,1,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0
290,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0


##### Converting DataFrame to numpy array for training

In [ ]:
x_features = data.to_numpy()

In [ ]:
x_features

array([['1', '1', '0', ..., 0, 0, 0],
       ['1', '1', '0', ..., 0, 0, 0],
       ['1', '1', '1', ..., 0, 0, 0],
       ...,
       ['1', '0', '1', ..., 0, 0, 0],
       ['1', '1', '1', ..., 0, 0, 0],
       ['0', '0', '1', ..., 0, 0, 0]], dtype=object)

In [ ]:
x_features.shape

(248, 21)

#### Split your data into training and test samples

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_features, y_labels, random_state=0,test_size=0.2)

In [ ]:
X_train

array([['1', '1', '0', ..., 1, 0, 0],
       ['1', '0', '1', ..., 0, 0, 0],
       ['1', '1', '1', ..., 0, 0, 0],
       ...,
       ['0', '1', '1', ..., 0, 1, 0],
       ['1', '0', '0', ..., 0, 0, 0],
       ['1', '1', '1', ..., 0, 0, 0]], dtype=object)

In [ ]:
X_train.shape

(198, 21)

In [ ]:
y_train

['0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0'

#### Build the Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

###### 600 trees in the random forest classifier with a max depth of 7 allowed for any tree 

In [ ]:
model = RandomForestClassifier(n_estimators=600,criterion="entropy",max_depth=7,random_state=42,verbose=1)

##### Train the model

In [ ]:
model.fit(X_train,y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    0.5s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=7, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=42, verbose=1,
                       warm_start=False)

##### Make Predictions on the test set

In [ ]:
y_pred = model.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:    0.0s finished


In [ ]:
y_pred

array(['0', '1', '1', '1', '0', '1', '1', '1', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0',
       '1', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0',
       '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1'], dtype='<U1')

#### Evaluate the model's performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(confusion_matrix(y_test,y_pred))

[[22  3]
 [ 1 24]]


In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92        25
           1       0.89      0.96      0.92        25

    accuracy                           0.92        50
   macro avg       0.92      0.92      0.92        50
weighted avg       0.92      0.92      0.92        50



In [136]:
print(accuracy_score(y_test, y_pred)*100,'%')

92.0 %
